# Text Preprocessing

In [ ]:
!pip install zemberek-python
!pip install antlr4-python3-runtime==4.9
!pip install unicode_tr

## Tokenize

In [ ]:
sample_doc = """
Mustafa Kemal Atatürk[c] (1881[d] - 10 Kasım 1938), Türk asker, devlet adamı ve Türkiye Cumhuriyeti'nin kurucusudur.

I. Dünya Savaşı sırasında Osmanlı ordusunda görev yapan Atatürk, Çanakkale Cephesi'nde miralaylığa, Sina ve Filistin Cephesi'nde ise Yıldırım Orduları komutanlığına atandı. Savaşın sonunda, Osmanlı İmparatorluğu'nun yenilgisini takiben 
Kurtuluş Savaşı ile simgelenen Türk Ulusal Hareketi'ne öncülük ve önderlik etti. 
Türk Kurtuluş Savaşı sürecinde Ankara Hükûmeti'ni kurdu, Türk Orduları Başkomutanı olarak Sakarya Meydan Muharebesi'ndeki başarısından dolayı 19 Eylül 1921 tarihinde "Gazi" unvanını aldı ve mareşallik rütbesine yükseldi. Askerî ve siyasi eylemleriyle İtilaf Devletleri ve destekçilerine karşı zafer kazandı. 

Savaşın ardından Cumhuriyet Halk Partisi'ni Halk Fırkası adıyla kurdu ve ilk genel başkanı oldu. 
29 Ekim 1923'te Cumhuriyetin İlanı akabinde Cumhurbaşkanı seçildi. 
1938'deki ölümüne dek dört dönem bu görevi yürüterek Türkiye'de en uzun süre cumhurbaşkanlığı yapmış kişi oldu. 


İstanbul, Isparta,
http://yavuzkomecoglu.com/, www.basakbuluz.com, #yâpayzeka, yaklaşık ~12',
e-posta adresim: komecoglu.yavuz@gmail.com
"""

sample_doc

### NLTK

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

#### sent_tokenize

In [ ]:
sentences = sent_tokenize(sample_doc, language='turkish')

for sentence in sentences:
  print(sentence)
  print("\n")

#### word_tokenize

In [ ]:
for sentence in sentences:
  print(sentence)
  words = word_tokenize(sentence)
  print(words)
  print("\n")

## Zemberek

In [ ]:
from zemberek import TurkishSentenceExtractor

#### zemberek - sentence tokenizer

In [ ]:
extractor = TurkishSentenceExtractor()
sentences = extractor.from_paragraph(sample_doc)

for sentence in sentences:
  print(sentence)
  print("\n")

#### zemberek - word tokenizer

In [ ]:
from zemberek import TurkishTokenizer
from zemberek.tokenization.token import Token

tokenizer = TurkishTokenizer.builder().accept_all().ignore_types(
    [Token.Type.NewLine, 
     Token.Type.SpaceTab,
     Token.Type.Punctuation]).build()


for sentence in sentences:
  print(sentence)
  tokens = tokenizer.tokenize(sentence)
  words = [token.content for token in tokens]
  print(words)
  print("\n")


In [ ]:
# TOKENIZATION
tokenizer = TurkishTokenizer.DEFAULT

tokens = tokenizer.tokenize("Saat 12:00.")
for token in tokens:
    print('Content = ', token.content)
    print('Type = ', token.type_.name)
    print('Start = ', token.start)
    print('Stop = ', token.end, '\n')

### lower(): Dökümanı küçük harfe çevirir

In [ ]:
# -*- coding: utf-8 -*-
from unicode_tr import unicode_tr

In [ ]:
sentences_lower = [sentence.lower() for sentence in sentences]
for sentence_lower in sentences_lower:
  print(sentence_lower)
  print("\n")

In [ ]:
sentences_lower = [unicode_tr(sentence).lower() for sentence in sentences]
for sentence_lower in sentences_lower:
  print(sentence_lower)
  print("\n")

### clean_word(): 

In [ ]:
import re
import string

def clean_word(word):
  unwanted_list = [u"&bull;", u"&lsquo;",u",",u"?",u"!",u'"',u"'",u"‘",u"’",u"/",u"<",u">",u"|",u"“",";","&","(",")","=","+","-","\\","*",":","~","@","."]
  alpha_list = [u"a",u"b",u"c",u"ç",u"d",u"e",u"f",u"g",u"ğ",u"h",u"ı",u"i",u"j",u"k",u"l",u"m",u"n",u"o",u"ö",u"p",u"q",u"r",u"s",u"ş",u"t",u"u",u"ü",u"v",u"w",u"x",u"y",u"z"," ",]

  # küçük harfe çeviriliyor
  word = unicode_tr(word).lower()
  txt = word.replace(u"â", u"a").lower()  # word.lower()

  # farklı karakterler siliniyor
  for unwanted_char in unwanted_list:
    # txt = txt.replace(uw,' ')
    # "Ankara'da" gibi tırnak kaldırıldığında "Ankara da" şeklinde oluşan aradaki boşluk birleştirildi.
    txt = txt.replace(unwanted_char, "")

  # geçerli harf listesi dışındaki harfler siliniyor
  chars = list(set(txt))
  for char in chars:
    if not char in alpha_list:
      txt = txt.replace(char, "")

  return txt

In [ ]:
clean_word("#yâpayzeka")

In [ ]:
for sentence in sentences:
  print(sentence)
  tokens = tokenizer.tokenize(sentence)
  for token in tokens:
    print(clean_word(token.content))
  print("\n")

### remove_URL: Dökümandan URL'leri kaldırın

In [ ]:
def remove_URL(text):
  # Remove URLs from a sample string
  pattern_url = r"""(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))"""
  text_clean = re.sub(pattern_url, "", text)
  
  return text_clean

In [ ]:
sentences_lower = [remove_URL(sentence) for sentence in sentences]
for sentence_lower in sentences_lower:
  print(sentence_lower)
  print("\n")

### remove_email: Dökümanlardan e-posta adreslerini 

In [ ]:
def remove_email(text):
  pattern_email = "\S*@\S*\s?"
  text_clean = re.sub(pattern_email, "", text)
  return text_clean


In [ ]:
sentences_lower = [remove_email(sentence) for sentence in sentences]
for sentence_lower in sentences_lower:
  print(sentence_lower)
  print("\n")

### NLTK - STOP WORDS

In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords_list = set(stopwords.words('turkish'))

In [ ]:
for sentence in sentences:
  print(sentence)
  tokens = tokenizer.tokenize(sentence)
  words = [token.content for token in tokens]
  print(words)

  print("words_filtered")
  words_filtered = []
  for w in words:
    if w not in stopwords_list:
      words_filtered.append(w)

  print(words_filtered)
  print("\n")


### Zemberek - Normalizer

In [ ]:
from zemberek import TurkishMorphology, TurkishSentenceNormalizer

examples = ["Yrn okua gidicem",
            "Tmm, yarin havuza giricem ve aksama kadar yaticam :)",
            "ah aynen ya annemde fark ettı siz evinizden cıkmayın diyo",
            "gercek mı bu? Yuh! Artık unutulması bile beklenmiyo",
            "Hayır hayat telaşm olmasa alacam buraları gökdelen dikicem.",
            "yok hocam kesınlıkle oyle birşey yok",
            "herseyi soyle hayatında olmaması gerek bence boyle ınsanların falan baskı yapıyosa",
            "email adresim komecoglu.yavuz@gmail.com",
            "Kredi başvrusu yapmk istiyrum.",
            "Bankanizin hesp blgilerini ogrenmek istyorum."]

morphology = TurkishMorphology.create_with_defaults()

normalizer = TurkishSentenceNormalizer(morphology)

In [ ]:
for example in examples:
    print(example)
    print(normalizer.normalize(example), "\n")

### Zemberek - SpellChecker

In [ ]:
from zemberek import TurkishSpellChecker

sc = TurkishSpellChecker(morphology)

In [ ]:
li = ["okuyablirim", "tartısıyor", "Ankar'ada", "knlıca", "yapablrim", "kıredi", "geldm", "geliyom", "aldm", "asln"]
start = time.time()
for word in li:
  print(word + " = " + ' '.join(sc.suggest_for_word(word)))

### Zemberek - MORPHOLOGICAL ANALYSIS

In [ ]:
# SINGLE WORD MORPHOLOGICAL ANALYSIS
results = morphology.analyze("kalemin")
for result in results:
    print(result)
print("\n")

In [ ]:
results = morphology.analyze("yazlık")
for result in results:
    print(result)
print("\n")

### Zemberek - Stemming

In [ ]:
word_analyze = morphology.analyze("görgüsüzlük")
if len(word_analyze.analysis_results) > 0:
  word_stem = word_analyze.analysis_results[0]
  print(word_stem.get_stem())
else:
  print(word)

In [ ]:
word_analyze = morphology.analyze("yazlık")
if len(word_analyze.analysis_results) > 0:
  word_stem = word_analyze.analysis_results[0]
  print(word_stem.get_stem())
else:
  print(word)